# Bilingual document translation

In this example we show an application of SONAR as a means of translation for study or language exploration.
We present the novel _The Metamorphosis_ by Franz Kafka in its English version, alongside the translation to German and Spanish.

In [1]:
%pip install sonar-space

from sonar.inference_pipelines.text import TextToTextModelPipeline, TextToEmbeddingModelPipeline

from sonar.models.sonar_text import (
    load_sonar_text_decoder_model,
    load_sonar_text_encoder_model,
    load_sonar_tokenizer,
)

import torch
device = torch.device("cpu")

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


/home/david/Documents/MLH_fellowship/SONAR/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
eng_sentences = ['One morning, as Gregor Samsa was waking up from anxious dreams, he discovered that in bed he had been changed into a monstrous verminous bug.', 'He lay on his armour-hard back and saw, as he lifted his head up a little, his brown, arched abdomen divided up into rigid bow-like sections.', 'From this height the blanket, just about ready to slide off completely, could hardly stay in place.', 'His numerous legs, pitifully thin in comparison to the rest of his circumference, flickered helplessly before his eyes.', '"What\'s happened to me," he thought.', 'It was no dream.', 'His room, a proper room for a human being, only somewhat too small, lay quietly between the four well-known walls.', 'Above the table, on which an unpacked collection of sample cloth goods was spread out (Samsa was a traveling salesman) hung the picture which he had cut out of an illustrated magazine a little while ago and set in a pretty gilt frame.', 'It was a picture of a woman with a fur hat and a fur boa.', 'She sat erect there, lifting up in the direction of the viewer a solid fur muff into which her entire forearm disappeared.', "Gregor's glance then turned to the window.", 'The dreary weather (the rain drops were falling audibly down on the metal window ledge) made him quite melancholy.', '"Why don\'t I keep sleeping for a little while longer and forget all this foolishness," he thought.', "But this was entirely impractical, for he was used to sleeping on his right side, and in his present state he couldn't get himself into this position.", 'No matter how hard he threw himself onto his right side, he always rolled again onto his back.', 'He must have tried it a hundred times, closing his eyes, so that he would not have to see the wriggling legs, and gave up only when he began to feel a light, dull pain in his side which he had never felt before.', '"O God," he thought, "what a demanding job I\'ve chosen!', 'Day in, day out on the road.', 'The stresses of trade are much greater than the work going on at head office, and, in addition to that, I have to deal with the problems of traveling, the worries about train connections, irregular bad food, temporary and constantly changing human relationships which never come from the heart.', 'To hell with it all!"', 'He felt a slight itching on the top of his abdomen.', 'He slowly pushed himself on his back closer to the bed post so that he could lift his head more easily, found the itchy part, which was entirely covered with small white spots (he did not know what to make of them), and wanted to feel the place with a leg.', 'But he retracted it immediately, for the contact felt like a cold shower all over him.', 'He slid back again into his earlier position.', '"This getting up early," he thought, "makes a man quite idiotic.', 'A man must have his sleep.', 'Other traveling salesmen live like harem women.', 'For instance, when I come back to the inn during the course of the morning to write up the necessary orders, these gentlemen are just sitting down to breakfast.', "If I were to try that with my boss, I'd be thrown out on the spot.", "Still, who knows whether that mightn't be really good for me.", "If I didn't hold back for my parents' sake, I would've quit ages ago.", "I would've gone to the boss and told him just what I think from the bottom of my heart.", "He would've fallen right off his desk!", 'How weird it is to sit up at the desk and talk down to the employee from way up there.', 'The boss has trouble hearing, so the employee has to step up quite close to him.', "Anyway, I haven't completely given up that hope yet.", "Once I've got together the money to pay off the parents' debt to him--that should take another five or six years--I'll do it for sure.", "Then I'll make the big break.", 'In any case, right now I have to get up.', 'My train leaves at five o\'clock."', 'And he looked over at the alarm clock ticking away by the chest of drawers.', '"Good God," he thought.', 'It was half past six, and the hands were going quietly on.', 'It was past the half hour, already nearly quarter to.', 'Could the alarm have failed to ring?', "One saw from the bed that it was properly set for four o'clock.", 'Certainly it had rung.', 'Yes, but was it possible to sleep through this noise that made the furniture shake?', "Now, it's true he'd not slept quietly, but evidently he'd slept all the more deeply.", 'Still, what should he do now?', "The next train left at seven o'clock.", 'To catch that one, he would have to go in a mad rush.', "The sample collection wasn't packed up yet, and he really didn't feel particularly fresh and active.", "And even if he caught the train, there was no avoiding a blow up with the boss, because the firm's errand boy would've waited for the five o'clock train and reported the news of his absence long ago.", "He was the boss's minion, without backbone or intelligence.", 'Well then, what if he reported in sick?', "But that would be extremely embarrassing and suspicious, because during his five years' service Gregor hadn't been sick even once.", "The boss would certainly come with the doctor from the health insurance company and would reproach his parents for their lazy son and cut short all objections with the insurance doctor's comments; for him everyone was completely healthy but really lazy about work.", 'And besides, would the doctor in this case be totally wrong?', 'Apart from a really excessive drowsiness after the long sleep, Gregor in fact felt quite well and even had a really strong appetite.']
spa_sentences = ['Cuando Gregorio Samsa se despertó una mañana después de un sueño intranquilo, se encontró sobre su cama convertido en un monstruoso insecto.', 'Estaba tumbado sobre su espalda dura, y en forma de caparazón y, al levantar un poco la cabeza, veía un vientre abombado, parduzco, dividido por partes duras en forma de arco, sobre cuya protuberancia apenas podía mantenerse el cobertor, a punto ya de resbalar al suelo.', 'Sus muchas patas, ridículamente pequeñas en comparación con el resto de su tamaño, le vibraban desamparadas ante los ojos.', '«¿Qué me ha ocurrido?», pensó.', 'No era un sueño.', 'Su habitación, una auténtica habitación humana, si bien algo pequeña, permanecía tranquila entre las cuatro paredes harto conocidas.', 'Por encima de la mesa, sobre la que se encontraba extendido un muestrario de paños desempaquetados – Samsa era viajante de comercio –, estaba colgado aquel cuadro, que hacía poco había recortado de una revista y había colocado en un bonito marco dorado.', 'Representaba a una dama ataviada con un sombrero y una boa” de piel, que estaba allí, sentada muy erguida y levantaba hacia el observador un pesado manguito de piel, en el cual había desaparecido su antebrazo.', 'La mirada de Gregorio se dirigió después hacia la ventana, y el tiempo lluvioso se oían caer gotas de lluvia sobre la chapa del alfeizar de la ventana – le ponía muy melancólico. « ¿Qué pasaría – pensó – si durmiese un poco más y olvidase todas las chifladuras?»', 'Pero esto era algo absolutamente imposible, porque estaba acostumbrado a dormir del lado derecho, pero en su estado actual no podía ponerse de ese lado.', 'Aunque se lanzase con mucha fuerza hacia el lado derecho, una y otra vez se volvía a balancear sobre la espalda.', 'Lo intentó cien veces, cerraba los ojos para no tener que ver las patas que pataleaban, y sólo cejaba en su empeño cuando comenzaba a notar en el costado un dolor leve y sordo que antes nunca había sentido.', '«Dios mío!», pensó.', '«¡Qué profesión tan dura he elegido!', 'Un día sí y otro también de viaje.', 'Los esfuerzos profesionales son mucho mayores que en el mismo almacén de la ciudad, y además se me ha endosado este ajetreo de viajar, el estar al tanto de los empalmes de tren, la comida mala y a deshora, una relación humana constantemente cambiante, nunca duradera, que jamás llega a ser cordial.', '¡Que se vaya todo al diablo!»', 'Sintió sobre el vientre un leve picor, con la espalda se deslizó lentamente más cerca de la cabecera de la cama para poder levantar mejor la cabeza; se encontró con que la parte que le picaba estaba totalmente cubierta por unos pequeños puntos blancos, que no sabía a qué se debían, y quiso palpar esa parte con una pata, pero inmediatamente la retiró, porque el roce le producía escalofríos.', 'Se deslizó de nuevo a su posición inicial.', '«Esto de levantarse pronto», pensó, «le hace a uno desvariar.', 'El hombre tiene que dormir.', 'Otros viajantes viven como pachás”.', 'Si yo, por ejemplo, a lo largo de la mañana vuelvo a la pensión para pasar a limpio los pedidos que he conseguido, estos señores todavía están sentados tomando el desayuno.', 'Eso podría intentar yo con mi jefe, en ese momento iría a parar a la calle.', 'Quién sabe, por lo demás, si no sería lo mejor para mí.', 'Si no tuviera que dominarme por mis padres, ya me habría despedido hace tiempo, me habría presentado ante el jefe y le habría dicho mi opinión con toda mi alma.', '¡Se habría caído de la mesa!', 'Sí que es una extraña costumbre la de sentarse sobre la mesa y, desde esa altura, hablar hacia abajo con el empleado que, además, por culpa de la sordera del jefe, tiene que acercarse mucho.', 'Bueno, la esperanza todavía no está perdida del todo; si alguna vez tengo el dinero suficiente para pagar las deudas que mis padres tienen con él – puedo tardar todavía entre cinco y seis años – lo hago con toda seguridad.', 'Entonces habrá llegado el gran momento, ahora, por lo pronto, tengo que levantarme porque el tren sale a las cinco», y miró hacia el despertador que hacía tictac sobre el armario.', '«¡Dios del cielo!», pensó.', 'Eran las seis y media y las manecillas seguían tranquilamente hacia delante, ya había pasado incluso la media, eran ya casi las menos cuarto.', '¿Es que no habría sonado el despertador?»', 'Desde la cama se veía que estaba correctamente puesto a las cuatro, seguro que también había sonado.', 'Sí, pero...', '¿Era posible seguir durmiendo tan tranquilo con ese ruido que hacía temblar los muebles?', 'Bueno, tampoco había dormido tranquilo, pero quizá tanto más profundamente.', '¿Qué iba a hacer ahora?', 'El siguiente tren salía a las siete, para cogerlo tendría que haberse dado una prisa loca, el muestrario todavía no estaba empaquetado, y él mismo no se encontraba especialmente espabilado y ágil; e incluso si consiguiese coger el tren, no se podía evitar una reprimenda del jefe, porque el mozo de los recados habría esperado en el tren de las cinco y ya hacía tiempo que habría dado parte de su descuido.', 'Era un esclavo del jefe, sin agallas ni juicio.', '¿Qué pasaría si dijese que estaba enfermo?', 'Pero esto sería sumamente desagradable y sospechoso, porque Gregorio no había estado enfermo ni una sola vez durante los cinco años de servicio.', 'Seguramente aparecería el jefe con el médico del seguro, haría reproches a sus padres por tener un hijo tan vago y se salvaría de todas las objeciones remitiéndose al médico del seguro, para el que sólo existen hombres totalmente sanos, pero con aversión al trabajo.', '¿Y es que en este caso no tendría un poco de razón?', 'Gregorio, a excepción de una modorra realmente superflua después del largo sueño, se encontraba bastante bien e incluso tenía mucha hambre.']
deu_sentences = ['Als Gregor Samsa eines Morgens aus unruhigen Träumen erwachte, fand er sich in seinem Bett zu einem ungeheueren Ungeziefer verwandelt.', 'Er lag auf seinem panzerartig harten Rücken und sah, wenn er den Kopf ein wenig hob, seinen gewölbten, braunen, von bogenförmigen Versteifungen geteilten Bauch, auf dessen Höhe sich die Bettdecke, zum gänzlichen Niedergleiten bereit, kaum noch erhalten konnte.', 'Seine vielen, im Vergleich zu seinem sonstigen Umfang kläglich dünnen Beine flimmerten ihm hilflos vor den Augen. »Was ist mit mir geschehen?« dachte er.', 'Es war kein Traum, sein Zimmer, ein richtiges, nur etwas zu kleines Menschenzimmer, lag ruhig zwischen den vier wohlbekannten Wänden, über dem Tisch, auf dem eine auseinandergepackte Musterkollektion von Tuchwaren ausgebreitet war - Samsa war Reisender -, hing das Bild, das er vor kurzem aus einer illustrierten Zeitschrift ausgeschnitten und in einem hübschen, vergoldeten Rahmen untergebracht hatte.', 'Es stellte eine Dame dar, die, mit einem Pelzhut und einer Pelzboa versehen, aufrecht dasaß und einen schweren Pelzmuff, in dem ihr ganzer Unterarm verschwunden war, dem Beschauer entgegenhob.', 'Gregors Blick richtete sich dann zum Fenster, und das trübe Wetter - man hörte Regentropfen auf das Fensterblech aufschlagen - machte ihn ganz melancholisch. »Wie wäre es, wenn ich noch ein wenig weiterschliefe und alle Narrheiten vergäße,« dachte er, aber das war gänzlich undurchführbar, denn er war gewöhnt, auf der rechten Seite zu schlafen, konnte sich aber in seinem gegenwärtigen Zustand nicht in diese Lage bringen.', 'Mit welcher Kraft er sich auch auf die rechte Seite warf, immer wieder schaukelte er in die Rückenlage [1178] zurück.', 'Er versuchte es wohl hundertmal, schloß die Augen, um die zappelnden Beine nicht sehen zu müssen und ließ erst ab, als er in der Seite einen noch nie gefühlten, leichten, dumpfen Schmerz zu fühlen begann. »Ach Gott,« dachte er, »was für einen anstrengenden Beruf habe ich gewählt!', 'Tag aus, Tag ein auf der Reise.', 'Die geschäftlichen Aufregungen sind viel größer, als im eigentlichen Geschäft zuhause, und außerdem ist mir noch diese Plage des Reisens auferlegt, die Sorgen um die Zuganschlüsse, das unregelmäßige, schlechte Essen, ein immer wechselnder, nie andauernder, nie herzlich werdender menschlicher Verkehr.', 'Der Teufel soll das alles holen!« Er fühlte ein leichtes Jucken oben auf dem Bauch; schob sich auf dem Rücken langsam näher zum Bettpfosten, um den Kopf besser heben zu können; fand die juckende Stelle, die mit lauter kleinen weißen Pünktchen besetzt war, die er nicht zu beurteilen verstand; und wollte mit einem Bein die Stelle betasten, zog es aber gleich zurück, denn bei der Berührung umwehten ihn Kälteschauer.', 'Er glitt wieder in seine frühere Lage zurück. »Dies frühzeitige Aufstehen«, dachte er, »macht einen ganz blödsinnig.', 'Der Mensch muß seinen Schlaf haben.', 'Andere Reisende leben wie Haremsfrauen.', 'Wenn ich zum Beispiel im Laufe des Vormittags ins Gasthaus zurückgehe, um die erlangten Aufträge zu überschreiben, sitzen diese Herren erst beim Frühstück.', 'Das sollte ich bei meinem Chef versuchen; ich würde auf der Stelle hinausfliegen.', 'Wer weiß übrigens, ob das nicht sehr gut für mich wäre.', 'Wenn ich mich nicht wegen meiner Eltern zurückhielte, ich hätte längst gekündigt, ich wäre vor den Chef hin getreten und hätte ihm meine Meinung von Grund des Herzens aus gesagt.', 'Vom Pult hätte er fallen müssen!', 'Es ist auch eine sonderbare Art, sich auf das Pult zu setzen und von der Höhe herab mit dem Angestellten zu reden, der überdies wegen der Schwerhörigkeit des Chefs ganz nahe herantreten muß.', 'Nun, die Hoffnung ist noch nicht gänzlich aufgegeben; habe ich einmal das Geld beisammen, um die Schuld der Eltern an ihn abzuzahlen - es dürfte noch fünf bis sechs Jahre dauern -, mache ich die Sache unbedingt.', 'Dann wird der große Schnitt gemacht.', 'Vorläufig allerdings muß ich aufstehen, denn mein Zug fährt um fünf.« Und er sah zur Weckuhr hinüber, die auf dem Kasten tickte. »Himmlischer Vater,« dachte er.', 'Es war halb sieben Uhr, und die Zeiger gingen ruhig vorwärts, es war sogar halb vorüber, es näherte sich schon dreiviertel.', 'Sollte der Wecker nicht geläutet haben?', 'Man sah vom Bett aus, daß er auf vier Uhr richtig eingestellt war; gewiß hatte er auch geläutet.', 'Ja, aber war es möglich, dieses möbelerschütternde Läuten ruhig zu verschlafen?', 'Nun, ruhig hatte er ja nicht geschlafen, aber wahrscheinlich desto fester.', 'Was aber sollte er jetzt tun?', 'Der nächste Zug ging um sieben Uhr; um den einzuholen, hätte er sich unsinnig beeilen müssen, und die Kollektion war noch nicht eingepackt, und er selbst fühlte sich durchaus nicht besonders frisch und beweglich.', 'Und selbst wenn er den Zug einholte, ein Donnerwetter des Chefs war nicht zu vermeiden, denn der Geschäftsdiener hatte beim Fünfuhrzug gewartet und die Meldung von seiner Versäumnis längst erstattet.', 'Es war eine Kreatur des Chefs, ohne Rückgrat und Verstand.', 'Wie nun, wenn er sich krank meldete?', 'Das war aber äußerst peinlich und verdächtig, denn Gregor war während seines fünfjährigen Dienstes noch nicht einmal krank gewesen.', 'Gewiß würde der Chef mit dem Krankenkassenarzt kommen, würde den Eltern wegen des faulen Sohnes Vorwürfe machen und alle Einwände durch den Hinweis auf den Krankenkassenarzt abschneiden, für den es ja überhaupt nur ganz gesunde, aber arbeitsscheue Menschen gibt.', 'Und hätte er übrigens in diesem Falle so ganz unrecht?', 'Gregor fühlte sich tatsächlich, abgesehen von einer nach dem langen Schlaf wirklich überflüssigen Schläfrigkeit, ganz wohl und hatte sogar einen besonders kräftigen Hunger.']

In [3]:
text_encoder_model = load_sonar_text_encoder_model("text_sonar_basic_encoder", device=device).eval()
text_decoder_model = load_sonar_text_decoder_model("text_sonar_basic_decoder", device=device).eval()
text_tokenizer = load_sonar_tokenizer("text_sonar_basic_encoder")

text_embedding_pipeline = TextToEmbeddingModelPipeline(text_encoder_model, text_tokenizer)

Using the cached checkpoint of text_sonar_basic_encoder. Set `force` to `True` to download again.
Using the cached checkpoint of text_sonar_basic_decoder. Set `force` to `True` to download again.
Using the cached tokenizer of text_sonar_basic_encoder. Set `force` to `True` to download again.


# Similarity of sentences between texts

We may compute the matrix cosine similarity of each of the sentences in the novel extracts, with each of the sentences present in the translations.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

eng_embeddings = text_embedding_pipeline.predict(eng_sentences, source_lang="eng_Latn")
spa_embeddings = text_embedding_pipeline.predict(spa_sentences, source_lang="spa_Latn")
deu_embeddings = text_embedding_pipeline.predict(deu_sentences, source_lang="deu_Latn")

eng_spa_cos_sim = cosine_similarity(eng_embeddings, spa_embeddings)
eng_deu_cos_sim = cosine_similarity(eng_embeddings, deu_embeddings)
spa_deu_cos_sim = cosine_similarity(spa_embeddings, deu_embeddings)

In [5]:
print(eng_deu_cos_sim)
print(eng_spa_cos_sim)
print(spa_deu_cos_sim)

[[ 0.72619224  0.08622358  0.06269339 ...  0.09275693  0.00628602
   0.22407693]
 [ 0.11609402  0.46587974  0.05548527 ...  0.07190079 -0.00443545
   0.06152777]
 [ 0.06567107  0.23206428  0.15597275 ...  0.06240445  0.15716389
   0.1589457 ]
 ...
 [ 0.04930421  0.13352732  0.09729137 ...  0.72410166  0.10511754
   0.09066909]
 [ 0.02775934 -0.05379335  0.06206077 ...  0.12726557  0.7053859
   0.091729  ]
 [ 0.2661551   0.11905408  0.12848586 ...  0.07357833  0.12886351
   0.67079294]]
[[ 0.682918    0.1001303   0.08201491 ...  0.10319082 -0.02233397
   0.19772843]
 [ 0.14484891  0.4911242   0.13205954 ...  0.08779293  0.01014944
   0.03730423]
 [ 0.08742855  0.2733525   0.17859717 ...  0.07594253  0.17223667
   0.14438803]
 ...
 [ 0.03904732  0.1515396   0.05158896 ...  0.6848761   0.03531618
   0.0910763 ]
 [ 0.03424467 -0.04957096  0.0783025  ...  0.12372334  0.5359933
   0.08312893]
 [ 0.26492468  0.14673     0.12761517 ...  0.04100559  0.10240456
   0.6001608 ]]
[[ 0.89043987  0.1

# Finding aligned sentence pairs

We may find which of the sentences correspond to which in the translation by determining the cosine distance and considering equivalent in meaning those sentences that are the closest together in the embedding space.

In [11]:
from fastdtw import fastdtw

eng_spa_distance, eng_spa_alignment = fastdtw(eng_embeddings, spa_embeddings, dist=cosine)
eng_deu_distance, eng_deu_alignment = fastdtw(eng_embeddings, deu_embeddings, dist=cosine)
spa_deu_distance, spa_deu_alignment = fastdtw(spa_embeddings, deu_embeddings, dist=cosine)

def print_aligned_sentences(alignment, sent1_list, sent2_list, lang1_name, lang2_name):
    print(f"Aligned Sentence Pairs ({lang1_name} -> {lang2_name}):")
    for idx1, idx2 in zip(alignment[0], alignment[1]):
        print(f"{lang1_name}: {sent1_list[idx1]}")
        print(f"{lang2_name}: {sent2_list[idx2]}")
        print("-" * 40)

print_aligned_sentences(eng_spa_alignment, eng_sentences, spa_sentences, "English", "Spanish")
print_aligned_sentences(eng_deu_alignment, eng_sentences, deu_sentences, "English", "German")
print_aligned_sentences(spa_deu_alignment, spa_sentences, deu_sentences, "Spanish", "German")


Aligned Sentence Pairs (English -> Spanish):
English: One morning, as Gregor Samsa was waking up from anxious dreams, he discovered that in bed he had been changed into a monstrous verminous bug.
Spanish: Estaba tumbado sobre su espalda dura, y en forma de caparazón y, al levantar un poco la cabeza, veía un vientre abombado, parduzco, dividido por partes duras en forma de arco, sobre cuya protuberancia apenas podía mantenerse el cobertor, a punto ya de resbalar al suelo.
----------------------------------------
English: One morning, as Gregor Samsa was waking up from anxious dreams, he discovered that in bed he had been changed into a monstrous verminous bug.
Spanish: Estaba tumbado sobre su espalda dura, y en forma de caparazón y, al levantar un poco la cabeza, veía un vientre abombado, parduzco, dividido por partes duras en forma de arco, sobre cuya protuberancia apenas podía mantenerse el cobertor, a punto ya de resbalar al suelo.
----------------------------------------
Aligned Sen

# Acknowledgements

We use texts from [Bilinguis.com](http://bilinguis.com), which has free books with their translations to different languages, along with some audio books as well.